# Voice input and output feature

In [11]:
import importlib
import utils
importlib.reload(utils)
from utils import get_agent

In [12]:
from utils import init_qa_chain, get_agent

qa_chain = init_qa_chain()
agent = get_agent(qa_chain)


In [ ]:
#voice input
# !pip install openai-whisper pyaudio pyttsx3
# !pip install sounddevice wavio

### Creates user audio recording functionality

In [15]:
import whisper
import sounddevice as sd
import numpy as np
import wavio
from gtts import gTTS
from playsound import playsound
import os
import uuid
import openai


import openai
import os

openai.api_key = os.getenv("OPENAI_API_KEY")


# Load Whisper model
model = whisper.load_model("base")

def record_audio(duration=5, fs=16000):
    print("🎙️ Recording...")
    recording = sd.rec(int(duration * fs), samplerate=fs, channels=1)
    sd.wait()
    wavio.write("/Users/test/Desktop/ironhack_labs/YouTube_ChatBot_Final/audio/user_input.wav", recording, fs, sampwidth=2)
    print("🎙️ Recording finished")

def transcribe_audio(filepath="/Users/test/Desktop/ironhack_labs/YouTube_ChatBot_Final/audio/user_input.wav"):
    print("📝 Transcribing...")
    result = model.transcribe(filepath)
    text = result["text"].strip()
    print(f"🗣️ You said: {text}")
    return text

def speak_text(text):
    filename = f"{uuid.uuid4().hex}.mp3"
    tts = gTTS(text=text, lang='en')
    tts.save(filename)
    playsound(filename)
    os.remove(filename)

def shorten_text(text, max_ratio=0.5):
    # Prompt the agent or LLM to summarize or shorten the text
    prompt = f"Please shorten the following text to about {int(max_ratio*100)}% of its length, keeping the key information:\n\n{text}"
    shortened = agent.run(prompt)
    return shortened

def answer_with_sources(question):
    # full_answer = agent.run(question)
    prompt = f"Use the YouTubeTranscriptQA tool to answer this question: {question}"
    full_answer = agent.run(prompt)
    short_answer = shorten_text(full_answer)
    return short_answer

# One interaction
def voice_rag_interaction():
    record_audio()
    question = transcribe_audio()
    answer = answer_with_sources(question)
    print(f"🤖 Answer:\n{answer}")
    speak_text(answer)


In [16]:
while True:
    voice_rag_interaction()
    cont = input("Ask another question? (y/n): ")
    if cont.strip().lower() != "y":
        break

🎙️ Recording...
🎙️ Recording finished
📝 Transcribing...


/opt/anaconda3/envs/cleanenv/lib/python3.10/site-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


🗣️ You said: What is the source for Bill 9?


> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: YouTubeTranscriptQA
Action Input: What is the source for Bill 9?

/Users/test/Desktop/ironhack_labs/YouTube_ChatBot_Final/notebooks/utils.py:65: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.



Observation: I'm sorry, but I don't have the information about the source for Bill 9.

Sources:
36 - Talking Aliens with NASA UAP Chair, David Spergel.en.txt
54 - Scientists Discuss Music and the Origins of Language.en.txt
59 - Spending 378 Days in a Mars Simulation with NASA Commander Kelly Haston.en.txt
13 - Kate the Chemist Crushes Your Chemistry Homework.en.txt
40 - Neil deGrasse Tyson and Bill Nye Catch Up.en.txt
Thought:Do I need to use a tool? No
AI: I'm sorry, but I don't have the information about the source for Bill 9.

> Finished chain.


> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? No
AI: Apologies, but I lack information on the source for Bill 9.

> Finished chain.
🤖 Answer:
Apologies, but I lack information on the source for Bill 9.
🎙️ Recording...
🎙️ Recording finished
📝 Transcribing...


/opt/anaconda3/envs/cleanenv/lib/python3.10/site-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


🗣️ You said: Who is Bill Nye?


> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: YouTubeTranscriptQA
Action Input: Who is Bill Nye?
Observation: Bill Nye is a popular American science communicator, television presenter, and mechanical engineer. He is best known for his television show "Bill Nye the Science Guy," which aimed to teach science to a young audience.

Sources:
40 - Neil deGrasse Tyson and Bill Nye Catch Up.en.txt
Thought:Do I need to use a tool? No
AI: Bill Nye is a renowned American science communicator, television presenter, and mechanical engineer. He gained popularity through his TV show "Bill Nye the Science Guy," which was designed to educate young people about science. This information is sourced from a transcript of a conversation between Neil deGrasse Tyson and Bill Nye.

> Finished chain.


> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? No
AI: Bill Nye, a famous American science communicator and TV host, i